In [ ]:
# from fiesc_big_data.ingest import ingest_sheets, save_csv, save_minio
from fiesc_big_data.transform import read_csv_from_minio

from minio import Minio
import pandas as pd

path_credentials = "fiesc_big_data/assents/fiesccase.json"
name_table = "Dados Educacionais Ficticios"

# Configurações do MinIO
minio_client = Minio(
    "localhost:9000",  # Endereço do servidor MinIO
    access_key="",  # Substitua pelo seu access key
    secret_key="",  # Substitua pelo seu secret key
    secure=False,  # True se estiver usando HTTPS
)

bucket_name = "fiesc"
pasta_destino = "bronze/nativos/"

file = read_csv_from_minio(
    minio_client, bucket_name, pasta_destino + "matricula_aulas.csv"
)

In [2]:
file.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 247291 entries, 0 to 247290
Data columns (total 4 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   id_matricula   247291 non-null  int64 
 1   id_disciplina  247291 non-null  int64 
 2   data_aula      247291 non-null  object
 3   presente       247291 non-null  bool  
dtypes: bool(1), int64(2), object(1)
memory usage: 5.9+ MB


In [68]:
import re
import unicodedata


def clear_text(texto):
    if isinstance(texto, str):
        # Remove acentos
        texto = (
            unicodedata.normalize("NFKD", texto)
            .encode("ASCII", "ignore")
            .decode("ASCII")
        )
        # Remove tudo que não for letra, número ou espaço
        texto = re.sub(r"[^A-Za-z0-9\s]", "", texto)
    return texto

In [60]:
# Split na única coluna (vai gerar 3 colunas separadas)
df = file.iloc[:, 0].str.split(";", expand=True)
df.columns = ["id_curso", "nome_curso", "area_conhecimento"]

In [71]:
# Aplica a função para todos os valores do DataFrame
df = df.applymap(clear_text)

/tmp/ipykernel_1689/606276797.py:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(clear_text)


In [14]:
import yaml

path = "fiesc_big_data/assents/config.yml"
# Carregar o arquivo YAML
with open(path) as file:
    config = yaml.safe_load(file)

In [22]:
from jinja2 import Environment, FileSystemLoader
import os


def render_sql_template(template_path, templates_dir="sql_templates"):
    # Configura o ambiente Jinja2
    env = Environment(
        loader=FileSystemLoader(templates_dir),
        autoescape=False,
        trim_blocks=True,
        lstrip_blocks=True,
    )
    # Carrega e renderiza o template
    template = env.get_template(template_path)
    return template.render()

ModuleNotFoundError: No module named 'jinja2'

In [20]:
config["minio"]["camadas"]["bronze"]["sheets"]["path"]

'bronze/sheets'

In [8]:
config["raw"]["cursos"]

'raw/cursos.csv'

In [ ]:
print("Extraindo Cursos")
cursos = ingest_sheets(path_credentials, name_table, 0)
print("Extraindo Disciplinas")
disciplinas = ingest_sheets(path_credentials, name_table, 1)
print("Extraindo Turmas")
turmas = ingest_sheets(path_credentials, name_table, 2)

save_csv(cursos, "cursos")
save_csv(disciplinas, "disciplinas")
save_csv(turmas, "turmas")

print("salvando dados do google sheets no minio")
bucket_name = "fiesc"
camada = "bronze"
fonte_dados_nativos = "nativos"
pasta_destino_nativos = f"{camada}/{fonte_dados_nativos}/"
fonte_sheets = "sheets"
pasta_destino = f"{camada}/{fonte_sheets}/"

# path google sheets
cursos_file = "fiesc_big_data/raw/cursos.csv"
disciplinas_file = "fiesc_big_data/raw/disciplinas.csv"
turmas_file = "fiesc_big_data/raw/turmas.csv"
# Path nativos
matricula_aula_file = "fiesc_big_data/dados/matricula_aulas.csv"
matricula_disciplinas_file = "fiesc_big_data/dados/matricula_disciplinas.csv"
matricula_notas_file = "fiesc_big_data/dados/matricula_notas.csv"
turma_avaliacoes_file = "fiesc_big_data/dados/turma_avaliacoes.csv"

save_minio(minio_client, bucket_name, pasta_destino, cursos_file)
save_minio(minio_client, bucket_name, pasta_destino, disciplinas_file)
save_minio(minio_client, bucket_name, pasta_destino, turmas_file)
print("Salvando dados nativos no minio")
save_minio(minio_client, bucket_name, pasta_destino_nativos, matricula_aula_file)
save_minio(minio_client, bucket_name, pasta_destino_nativos, matricula_disciplinas_file)
save_minio(minio_client, bucket_name, pasta_destino_nativos, matricula_notas_file)
save_minio(minio_client, bucket_name, pasta_destino_nativos, turma_avaliacoes_file)


df_situacao_notas = read_spark_delta(spark, path_situacao_notas)
df_situacao_notas.createOrReplaceTempView("situacao_notas")

df_situacao_presenca = read_spark_delta(spark, path_situacao_presenca)
df_situacao_presenca.createOrReplaceTempView("situacao_presenca")

# Exemplo: ler dados de uma tabela
df_alunos = spark.read.jdbc(url=db_url, table="alunos", properties=properties)

df_alunos_idade = df_alunos.withColumn(
    "idade", floor(datediff(current_date(), col("data_nascimento")) / 365.25)
).withColumn(
    "idade_valida",
    when((col("idade") >= 18) & (col("idade") <= 90), "sim").otherwise("não"),
)
df_alunos_idade.createOrReplaceTempView("alunos")

In [ ]:
# from fiesc_big_data.ingest import ingest_sheets, save_csv, save_minio
from fiesc_big_data.transform import read_csv_from_minio, clear_text, delta_lake_minio
from minio import Minio
import pandas as pd

path_credentials = "fiesc_big_data/assents/config/fiesccase.json"
name_table = "Dados Educacionais Ficticios"

# Configurações do MinIO
minio_client = Minio(
    "localhost:9000",  # Endereço do servidor MinIO
    access_key="minioadmin",  # Substitua pelo seu access key
    secret_key="minioadmin123",  # Substitua pelo seu secret key
    secure=False,  # True se estiver usando HTTPS
)

bucket_name = "fiesc"
pasta_destino = "bronze/sheets/"

df_cursos = read_csv_from_minio(minio_client, bucket_name, pasta_destino + "cursos.csv")
df_cursos = df_cursos.iloc[:, 0].str.split(";", expand=True)
df_cursos.columns = ["id_curso", "nome_curso", "area_conhecimento"]
df_cursos = df_cursos.applymap(clear_text)

df_disciplinas = read_csv_from_minio(
    minio_client, bucket_name, pasta_destino + "disciplinas.csv"
)
df_disciplinas = df_disciplinas.iloc[:, 0].str.split(";", expand=True)
df_disciplinas.columns = [
    "id_disciplina",
    "id_curso",
    "nome_disciplina",
    "sigla",
    "dia_semana",
    "carga_horaria",
    "avaliacao",
]
df_disciplinas["carga_horaria"] = df_disciplinas["carga_horaria"].astype(int)
df_disciplinas["avaliacao"] = df_disciplinas["avaliacao"].replace("", "0").astype(int)
df_disciplinas = df_disciplinas.applymap(clear_text)


df_turmas = read_csv_from_minio(minio_client, bucket_name, pasta_destino + "turmas.csv")
df_turmas = df_turmas.iloc[:, 0].str.split(";", expand=True)
df_turmas.columns = [
    "id_turma",
    "nome_turma",
    "data_inicio",
    "data_fim",
    "codigo_curso",
]
df_turmas["data_inicio"] = pd.to_datetime(df_turmas["data_inicio"], format="%Y-%m-%d")
df_turmas["data_fim"] = pd.to_datetime(df_turmas["data_fim"], format="%Y-%m-%d")
df_turmas = df_turmas.applymap(clear_text)

# Lendo dados nativos do MinIO
pasta_minio = "bronze/nativos/"
df_matricula_aula = read_csv_from_minio(
    minio_client, bucket_name, pasta_minio + "matricula_aulas.csv"
)

df_matriculas_disciplina = read_csv_from_minio(
    minio_client, bucket_name, pasta_minio + "matricula_disciplinas.csv"
)

df_matriculas_notas = read_csv_from_minio(
    minio_client, bucket_name, pasta_minio + "matricula_notas.csv"
)

df_turma_avaliacao = read_csv_from_minio(
    minio_client, bucket_name, pasta_minio + "turma_avaliacoes.csv"
)

# Salvando os dados no em Delta Lake MinIO

camada = "silver"
print("Salvando os dados no Delta Lake Cursos")
delta_lake_minio(minio_client, bucket_name, camada, "cursos", df_cursos)
print("Salvando os dados no Delta Lake Disciplinas")
delta_lake_minio(minio_client, bucket_name, camada, "disciplinas", df_disciplinas)
print("Salvando os dados no Delta Lake Turmas")
delta_lake_minio(minio_client, bucket_name, camada, "turmas", df_turmas)
print("Salvando os dados no Delta Lake Matricula Aulas")
delta_lake_minio(
    minio_client, bucket_name, camada, "matricula_aulas", df_matricula_aula
)
print("Salvando os dados no Delta Lake Matricula Disciplinas")
delta_lake_minio(
    minio_client, bucket_name, camada, "matricula_disciplinas", df_matriculas_disciplina
)
print("Salvando os dados no Delta Lake Matricula Notas")
delta_lake_minio(
    minio_client, bucket_name, camada, "matricula_notas", df_matriculas_notas
)
print("Salvando os dados no Delta Lake Turma Avaliacoes")
delta_lake_minio(
    minio_client, bucket_name, camada, "turma_avaliacoes", df_turma_avaliacao
)
print("✔️ Dados transformados e salvos com sucesso no Delta Lake.")